In [1]:
from __future__ import annotations

from torch.utils.data import random_split

from mmpfn.datasets.salary import SalaryDataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score
from mmpfn.models.mmpfn_v2 import MMPFNClassifier
from mmpfn.models.mmpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.mmpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune_mm.finetune_tabpfn_main import fine_tune_tabpfn

/home/wall/anaconda3/envs/mmpfn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # for debugging

In [3]:
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/salary")
dataset = SalaryDataset(data_path)
_ = dataset.get_embeddings()

Load embeddings from embeddings/salary/salary.pt


In [4]:
accuracy_scores = []
for seed in range(5):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    text_train = train_dataset.dataset.embeddings[train_dataset.indices].unsqueeze(1)
    text_test = test_dataset.dataset.embeddings[test_dataset.indices].unsqueeze(1)
        
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_mmpfn_airbnb.ckpt"

    fine_tune_tabpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=pd.DataFrame(X_train),
        image_train=text_train,
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
        freeze_input=True,  # Freeze the input layers (encoder and y_encoder) during finetuning
        mixer_type='MGM' # MGM MGM+CQAM
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = MMPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
        mixer_type='MGM' # MGM MGM+CQAM
    )

    clf_finetuned = model_finetuned.fit(X_train, text_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(X_test, text_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:   4%|▍         | 4/100 [00:24<12:00,  7.51s/it, Best Val. Loss=1.33, Best Val. Score=-1.33, Training Loss=1.3, Val. Loss=1.33, Patience=47, Utilization=0, Grad Norm=1.95] [2025-09-09 12:59:58,510] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  99%|█████████▉| 99/100 [10:27<00:06,  6.36s/it, Best Val. Loss=1.3, Best Val. Score=-1.3, Training Loss=1.24, Val. Loss=1.3, Patience=-47, Utilization=0, Grad Norm=2.12]  [2025-09-09 13:10:01,867] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [10:38,  6.39s/it, Best Val. Loss=1.3, Best Val. Score=-1.3, Training Loss=1.26, Val. Loss=1.3, Patience=-48, Utilization=0, Grad Norm=2.74]                         
[2025-09-09 13:10:08,256] INFO - Initial Validation Loss: 1.3392215386815003 Best Validation Loss: 1.2976129167651838 Total Steps: 101 Best Step: 100 Total Time Spent: 642.3699605464935
/home/wall/works/research/MultiModalPFN/mmpfn/models/mmpfn_v

accuracy_score (Finetuned): 0.437263317344105


Fine-tuning Steps:   6%|▌         | 6/100 [00:31<09:59,  6.37s/it, Best Val. Loss=1.31, Best Val. Score=-1.31, Training Loss=1.31, Val. Loss=1.31, Patience=45, Utilization=0, Grad Norm=1.83][2025-09-09 13:10:55,810] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  84%|████████▍ | 84/100 [08:45<01:41,  6.33s/it, Best Val. Loss=1.29, Best Val. Score=-1.29, Training Loss=1.26, Val. Loss=1.29, Patience=-32, Utilization=0, Grad Norm=2.41][2025-09-09 13:19:09,463] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [10:31,  6.32s/it, Best Val. Loss=1.29, Best Val. Score=-1.29, Training Loss=1.26, Val. Loss=1.29, Patience=-48, Utilization=0, Grad Norm=3.07]                         
[2025-09-09 13:20:50,738] INFO - Initial Validation Loss: 1.336951353706309 Best Validation Loss: 1.2883711138380742 Total Steps: 101 Best Step: 100 Total Time Spent: 633.393468618393
/home/wall/works/research/MultiModalPFN/mmpfn/models/mmpfn

accuracy_score (Finetuned): 0.42085331986872004


Fine-tuning Steps:  14%|█▍        | 14/100 [01:23<09:08,  6.38s/it, Best Val. Loss=1.31, Best Val. Score=-1.31, Training Loss=1.29, Val. Loss=1.31, Patience=37, Utilization=0, Grad Norm=1.7] [2025-09-09 13:22:28,374] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [10:35,  6.35s/it, Best Val. Loss=1.3, Best Val. Score=-1.3, Training Loss=1.25, Val. Loss=1.3, Patience=-49, Utilization=0, Grad Norm=2.19]                          
[2025-09-09 13:31:35,460] INFO - Initial Validation Loss: 1.3286779900335302 Best Validation Loss: 1.2963168058377652 Total Steps: 101 Best Step: 100 Total Time Spent: 636.6358184814453
/home/wall/works/research/MultiModalPFN/mmpfn/models/mmpfn_v2/classifier.py:400: UserWarning: Number of samples 15841 is greater than the maximum Number of samples 10000 supported by the model. You may see degraded performance.
  X, y, feature_names_in, n_features_in = validate_Xy_fit(


accuracy_score (Finetuned): 0.4236303963645544


Fine-tuning Steps:   6%|▌         | 6/100 [00:31<10:00,  6.38s/it, Best Val. Loss=1.33, Best Val. Score=-1.33, Training Loss=1.3, Val. Loss=1.33, Patience=45, Utilization=0, Grad Norm=1.47] [2025-09-09 13:32:21,861] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [10:36,  6.36s/it, Best Val. Loss=1.3, Best Val. Score=-1.3, Training Loss=1.29, Val. Loss=1.3, Patience=-49, Utilization=0, Grad Norm=2]                              
[2025-09-09 13:42:21,052] INFO - Initial Validation Loss: 1.3438833303457924 Best Validation Loss: 1.3039206347959382 Total Steps: 101 Best Step: 97 Total Time Spent: 637.6108298301697
/home/wall/works/research/MultiModalPFN/mmpfn/models/mmpfn_v2/classifier.py:400: UserWarning: Number of samples 15841 is greater than the maximum Number of samples 10000 supported by the model. You may see degraded performance.
  X, y, feature_names_in, n_features_in = validate_Xy_fit(


accuracy_score (Finetuned): 0.41327947488008077


Fine-tuning Steps:   7%|▋         | 7/100 [00:38<09:54,  6.39s/it, Best Val. Loss=1.3, Best Val. Score=-1.3, Training Loss=1.33, Val. Loss=1.3, Patience=44, Utilization=0, Grad Norm=1.55]   [2025-09-09 13:43:13,967] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  64%|██████▍   | 64/100 [06:47<03:58,  6.61s/it, Best Val. Loss=1.27, Best Val. Score=-1.27, Training Loss=1.29, Val. Loss=1.27, Patience=-12, Utilization=0, Grad Norm=1.98][2025-09-09 13:49:22,870] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [10:43,  6.43s/it, Best Val. Loss=1.27, Best Val. Score=-1.27, Training Loss=1.27, Val. Loss=1.27, Patience=-48, Utilization=0, Grad Norm=1.92]                         
[2025-09-09 13:53:13,803] INFO - Initial Validation Loss: 1.3207194438352474 Best Validation Loss: 1.2697563579597513 Total Steps: 101 Best Step: 99 Total Time Spent: 644.7358176708221
/home/wall/works/research/MultiModalPFN/mmpfn/models/mmpf

accuracy_score (Finetuned): 0.4329714718505428


In [5]:
X_train[0].shape

(3,)

In [6]:
text_train[0].shape

torch.Size([1, 768])

In [7]:
print(len(X_train))
print(len(text_train))


15841
15841


In [8]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.4255995960616006
Std Accuracy: 0.00858576983009472
